# Taks 2

In [7]:
# importing Image class from PIL package  
from PIL import Image  
import psycopg2

# ER Diagram 

![ER Diagram](https://i.imgur.com/naRsMrw.png)


Above, we have created and imported our schema to our python notebook as an image. We have made the decision to exclude 'meta_keyword', 'type, 'domain', 'authors' and 'tag' as 
attributes of 'article', and instead have their own tables. This was done, to create the necessary entity-relations. For example, in the table 'type' there are 11 types, that each has their own id because 'type' is a one-to-many relation, and that id
is saved in the 'article' instance. 'tag' on the other hand is a many-to-many relation, and we have therefore created a relation table that saves the tags associated with an article. In this 
relation table the id of the article and the tag is saved together. The other one-to-many and many-to-many relations works as described above.The SQL-code that was used to create the database can be found in the file "SQL_database.sql". 

The code used to separate the database and clean the data can be seen in the files 'Organize.ipynb' and 'Clean.ipynb' respectively. Here 'Clean.ipynb' cleans the content of the 
articles, and 'Organize.ipynb' separates the articles into different csv-files which we populate our database with. 


In [8]:
def execQuery(query):
    try:
        connection = psycopg2.connect(user = "postgres",
                                      password = "detminkode",
                                      host = "localhost",
                                      port = "5432",
                                      database = "postgres")
        cursor = connection.cursor()
        cursor.execute(query)
        record = cursor.fetchall()
        return record
    except (Exception, psycopg2.Error) as error :
        connection = False
        print ("Error while connecting to PostgreSQL", error)
    finally:
        if(connection):
            cursor.close()
            connection.close()
            print("Executed query and closed connection.")

In [10]:
execQuery("""Select Count(id)
from article""")

Executed query and closed connection.


[(991691,)]

# Task 3

## 3.1

To solve task 3.1, we have created two queries, one with and one without INNER JOIN. We save our 'scraped_at' dates as 'Dates', to utilize the >= operator to find the articles that
has been scraped after 15. Jan 2018. We also isolated the articles with type_id = 4, as this is the id for the articles classified as 'reliable'.

In [ ]:
SELECT domain_name.domain_name FROM domain_name
WHERE domain_id in (SELECT article.domain_id FROM article
WHERE article.type_id = '4' and scraped_at >= '2018-01-15')

Without INNER JOIN above, and without it below

In [ ]:
SELECT DISTINCT domain_name.domain_name FROM domain_name
INNER JOIN article
ON article.domain_id = domain_name.domain_id
WHERE article.type_id = '4' and scraped_at >= '2018-01-15'

In [4]:
execQuery("""SELECT DISTINCT domain_name.domain_name FROM domain_name
INNER JOIN article
ON article.domain_id = domain_name.domain_id
WHERE article.type_id = '12' and scraped_at >= '2018-01-15'""")

Executed query and closed connection.


[('christianpost.com',), ('consortiumnews.com',), ('nutritionfacts.org',)]

## 3.2

With the query below we find the author(s) with the most articles classified as fake news. This is done by finding all the authors for the articels that have been classified as fake, then count the authors, and lastly group all the authors that have the same number of fake articels. This is to make sure that if two or more authors have shared the most artikels. Then we order them and take first element which will be the largest number.

In [ ]:
SELECT array_agg(authorname), counter
FROM (SELECT COUNT(author_in.authorid) as counter, author.authorname
    FROM author_in
    INNER JOIN author
    ON author.authorid = author_in.authorid
    WHERE Author.authorname != 'NoAuthor' AND author_in.id in (SELECT article.id FROM article WHERE article.type_id = 7)
    GROUP BY author_in.authorid, author.authorname) AS authors
GROUP BY counter
ORDER BY counter DESC
limit 1

In [6]:
execQuery("""SELECT array_agg(authorname), counter
FROM (SELECT COUNT(author_in.authorid) as counter, author.authorname
    FROM author_in
    INNER JOIN author
    ON author.authorid = author_in.authorid
    WHERE Author.authorname != 'NoAuthor' AND author_in.id in (SELECT article.id FROM article WHERE article.type_id = 7)
    GROUP BY author_in.authorid, author.authorname) AS authors
GROUP BY counter
ORDER BY counter DESC
limit 1 """)

Executed query and closed connection.


[(['John Rolls'], 1142)]

## 3.3

The method we used to count the pairs, was to compare 3 articles (a, b, c) and if they had the same meta_tags, to then create the pairs (a, b) and (a, c).
We do not include the pair (b, c), as this information can be deduced by the other pairs. Therefore the number of pairs are Count(*) - 1, as can be seen 
on line 2 in the query. The way it works, is by collapsing all the meta_tags with the articles, so the meta_tags get saved as a list instead of all the 
different rows of data. We do this with the aggregate function 'array_agg'. After this we collapse the article id's that has the same list of meta_tags
using the same 'array_agg' fuction, and if the length of the list is more than one then there are one or more pairs. Lastly, we then sum all the pairs,
which returns the total number of article pairs with the same meta_tags.

In [ ]:
SELECT sum(pairAmount)
FROM (SELECT count(*) - 1 as pairAmount
	FROM (SELECT id, array_agg(meta_keywords.meta_keyword) AS Meta_Keyword_ids
			FROM meta_keywords_in
			INNER JOIN meta_keywords
			ON meta_keywords.meta_keyword_id = meta_keywords_in.meta_keyword_id
			WHERE meta_keywords.meta_keyword_id != '0'
			GROUP BY meta_keywords_in.id
		 ) meta_keywords_in
	GROUP BY Meta_Keyword_ids
	HAVING count(*) > 1) AS counter

In [8]:
execQuery("""SELECT sum(pairAmount)
FROM (SELECT count(*) - 1 as pairAmount
	FROM (SELECT id, array_agg(meta_keywords.meta_keyword) AS Meta_Keyword_ids
			FROM meta_keywords_in
			INNER JOIN meta_keywords
			ON meta_keywords.meta_keyword_id = meta_keywords_in.meta_keyword_id
			WHERE meta_keywords.meta_keyword_id != '0'
			GROUP BY meta_keywords_in.id
		 ) meta_keywords_in
	GROUP BY Meta_Keyword_ids
	HAVING count(*) > 1) AS counter""")

Executed query and closed connection.


[(Decimal('24773'),)]

# Assignment 4

## Dataexploration Query 1

In [9]:
execQuery("""SELECT sum(length(content))/COUNT(id) as AvgLen, type
from article
Inner join type
ON article.type_id = type.type_id
GROUP BY type
ORDER BY AvgLen ASC """)

Executed query and closed connection.


[(1510, 'satire'),
 (1629, 'rumor'),
 (1914, 'unreliable'),
 (2399, 'conspiracy'),
 (2459, None),
 (2460, 'clickbait'),
 (3060, 'junksci'),
 (3072, 'fake'),
 (3501, 'political'),
 (3543, 'unknown'),
 (3667, 'bias'),
 (4365, 'reliable'),
 (8426, 'hate')]

## Dataexploration Query 2

In [18]:
execQuery("""SELECT COUNT(meta_keywords_in.meta_keyword_id), meta_keywords.meta_keyword
FROM meta_keywords_in
inner join
article
ON article.id = meta_keywords_in.id and article.type_id = 7
inner join 
meta_keywords
ON meta_keywords.meta_keyword_id = meta_keywords_in.meta_keyword_id
GROUP BY meta_keywords.meta_keyword
Having COUNT(meta_keywords_in.meta_keyword_id) > 1
ORDER BY count DESC""")

Executed query and closed connection.


[(123512, None),
 (5, 'texas'),
 (3, 'donald trump'),
 (3, 'Kfc'),
 (2, 'Clinton Foundation'),
 (2, 'president obama'),
 (2, 'secession'),
 (2, 'spontaneous combustion'),
 (2, 'trump'),
 (2, 'hillary clinton'),
 (2, 'Kentucky Fried Chicken'),
 (2, 'fox news'),
 (2, 'fried chicken')]

## Dataexploration Query 3

In [20]:
execQuery("""SELECT sum(length(tags.tag)) / count(*) as Avg, type.type
FROM tags_in
INNER JOIN article
ON article.id = tags_in.id
inner join tags
ON tags.tag_id = tags_in.tag_id
inner join type
ON type.type_id = article.type_id
GROUP BY type.type
ORDER BY Avg DESC""")

Executed query and closed connection.


[(43, 'rumor'),
 (18, None),
 (17, 'bias'),
 (17, 'hate'),
 (16, 'fake'),
 (16, 'political'),
 (16, 'conspiracy'),
 (16, 'unknown'),
 (16, 'clickbait'),
 (15, 'unreliable'),
 (14, 'junksci'),
 (12, 'satire'),
 (12, 'reliable')]

# Task 5

I this task we had to make a web crawler that could scrape information from "Politics and Conflict" on Wikinews and based on our group number we would should only select a subset of all articles. This was given to us in the form of some python code to generate a string with the beginning letter for articles within our subset of articles.

The first step we took was to figure out what kind of task we were given before we decided on a given tool.

The first few observations we made were regarding how the webpage indexed its articles such that we could make our crawling logic.

The first observation we made was that each "entry point" for a given letter was easy to get as it was just https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from=[letter], where the letter was at the end of the url. on a given entry point there are links to categories and articles however there are a maximum 200 pages, even if there are more articles with a given starting letter, so we had to follow links to find all articles. A sticking point arose when we looked at "indexing"-urls after our entry points as they had no information of what letter/entry point we were coming from as they where of the form https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&pagefrom=[start article]+[end article], such that the url only described by article tiles what other articles where on that indexing page. This mean that our tool had to be able to follow links and have knowledge of what page it was coming from as it could not just use information on a page as well as jumping to all articles on an indexing page.

Next we looked at a few articles and they seemed to have a general structure such that locating them on a page within HTML would be somewhat doable so we postponed the actual scraping part for later.

Because of the requirements based on our initial assesment of the problem and our look at python web scraping tools we choose Scrapy as it is a feature rich tool made for making web crawlers and as we had some requirements that where non trivial ie. traversal logic more complicated then get all links and so on we choose it.

When implementing our scraper, we encountered a lot obstacles along the way, amplified by the fact that we assigned tasks such that people with less explerience in an given subject had to do it for our assignment.

We started by reading documentation while watching and reading tutorials as we building a dummy spider for a smaller part of the problem to get the basics of Scrapy right as well as understading HTML-markup node navigation. We made use of xpath to locate nodes within HTML. A snapshot of our Scrapy class we made is given below along with its output:




In [3]:
import pandas as pd


In [40]:
filepath = '/home/daniel/OneDrive/KUuni/DataScience/Python/DS_5/wiki_news_nr_12.csv'

# we read in the file
df = pd.read_csv(filepath)

# print shape -< how many rows and elements it has
print(df.shape, "<- shape of wikinews-dataset(row, col)")

# how many unique urls did we get(optimally as many as the rows in our dataframe)
unique_articles = df['article_url'].unique().shape
print(unique_articles, "<- num of unique articles - seems to have the same number of rows no duplicates")

# how many fields without daata ie. nan out of all fiels
df_nan_elms = df.isna().sum().sum()
df_not_nan_elms = df.notna().sum().sum()
print( df_nan_elms, "<- num of nan entries out of ->",  df_not_nan_elms ,
     "\nnan rate of", df_nan_elms/df_not_nan_elms, "%")
# majority comes form articles not having sources or source wiki pages
print(df['sources_url'].isna().sum(), "nan elements from sources_url column")
print( df['about_sources_wiki_url'].isna().sum(), "nan elements from about_sources_wiki_url column\n")
print(df.info())

(3680, 8) <- shape of wikinews-dataset(row, col)
(3680,) <- num of unique articles - seems to have the same number of rows no duplicates
437 <- num of nan entries out of -> 29003 
nan rate of 0.01506740681998414 %
216 nan elements from sources_url column
210 nan elements from about_sources_wiki_url column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3680 entries, 0 to 3679
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   article_url             3680 non-null   object
 1   categories              3680 non-null   object
 2   content                 3678 non-null   object
 3   publish_date            3671 non-null   object
 4   scraped_at              3680 non-null   object
 5   sources_url             3464 non-null   object
 6   about_sources_wiki_url  3470 non-null   object
 7   title                   3680 non-null   object
dtypes: object(8)
memory usage: 230.1+ KB
None


This test spider below was used to see of we could iterate over a list of article-URLs if we could locate them on a navigation page. This early spider only has 2 parts; one for iterating over URLs and the other for getting article scraping data. The spider above worked fine, but the next step being creating the navigation logic became the greatest challenge, caused by a simple thing. We wrote the spider logic for another website as it was simpler and then adapted it to wikinews, however when we interchanged our other website with wikinews no files where generated when we selected to get output. After debugging we came across an Error about robot.txt which tunrns out is used when accesing websites from non-intruductionary-tutorials as it is a policy obayed by all never Scrapy spider by default if a website dosen't allow certain kinds of scrapers. After this we changed our spider to not obey the robot.txt but read https://en.wikipedia.org/robots.txt instead, and implimented some restrictions on the spider. Below the test spider we have included some of the settings we enabled to scrape more responsibly


In [ ]:
``` Test spider - bash
class testSpider(scrapy.Spider):
    name = "test"
    def start_requests(self):
        urls = [
            'https://en.wikinews.org/wiki/A_policeman_is_killed_and_another_one_is_tortured_in_MST_camp,_in_Brazil',

            'https://en.wikinews.org/wiki/African_Union_refuses_to_arrest_Sudan%27s_President_for_war_crimes',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    # get article content
    def parse(self, response):
        for info in response.xpath('//div[@id="content"]'):
            yield {
                'title': info.xpath('//*[@id="firstHeading"]/text()').get(),
            }



``` Settings - python
AUTOTHROTTLE_ENABLED = True

# The initial download delay
AUTOTHROTTLE_START_DELAY = 5

# The maximum download delay to be set in case of high latencies
AUTOTHROTTLE_MAX_DELAY = 30

# The average number of requests Scrapy should be sending in parallel to
# each remote server

AUTOTHROTTLE_TARGET_CONCURRENCY = 1.0
# Enable showing throttling stats for every response received:

# Enable and configure HTTP caching (disabled by default)
HTTPCACHE_ENABLED = True



Our final spider became somewhat complicated for a first spider and crawls without any issues.


In [ ]:
``` python
import string
import re
import scrapy
from scrapy.loader import ItemLoader
from scrapy.loader.processors import Join, Compose
from datetime import datetime
from urllib.parse import urljoin
from ..items import articleItem # location of item - used for scraped data structure

# creating urls for chars based on group _nr - change group_nr to generate start_urls
group_nr = 12 # <- change to get correct article set
urls =[]
for char in "ABCDEFGHIJKLMNOPRSTUVWZABCDEFGHIJKLMNOPRSTUVWZ"[group_nr % 23:group_nr % 23+10]:
    urls.append('https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from=' + char)
print(*urls, sep='\n')

# main spider
class wikiSpider(scrapy.Spider):
    name = "wiki"

    # start urls for scraping
    def start_requests(self):

        # urls used to spawn spider-instances
        global urls
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    # Set the maximum depth
    maxdepth = 10

    def parse(self, response):
        """ Main method that parse downloaded pages. """
        # Set defaults for the first page that won't have any meta information
        start_url = ''
        from_url = ''
        from_text = ''
        depth = 0
        # Extract the meta information from the response, if any
        if 'start'  in response.meta: start_url = response.meta['start']
        if 'from'   in response.meta: from_url  = response.meta['from']
        if 'text'   in response.meta: from_text = response.meta['text']
        if 'depth'  in response.meta: depth     = response.meta['depth']
        
        # set start url for crawler
        if depth == 0:
            start_url = response.url

        # get all article links
        if start_url[-1] == response.xpath('//div[@id="mw-pages"]/div/div/div[1]/h3/text()').get(): # chek that current letter is on page

            # change xpath to: ('//div[@id="mw-pages"]/div/div/div[1]/ul/li[1]/a/@href') <- 1   page
            # change xpath to: ('//div[@id="mw-pages"]/div/div/div[1]/ul/li/a/@href')    <- 200 pages
            articles = response.xpath('//div[@id="mw-pages"]/div/div/div[1]/ul/li/a/@href').getall()
            for a in articles:
                url = urljoin(response.url, a)
                yield scrapy.Request(url, callback=self.parse_article)

        ### DEBUG printing - used for locating spider behavior ###
        print("### DEBUG DUMP STEP:", depth, response.url, '<-', from_url, from_text, "END ###",
              "### DEBUG DUMP start_url:", start_url[-1], response.xpath('//div[@id="mw-pages"]/div/div/div[1]/h3/text()').get(),"char_page END ###")

        # get nex_page only if maximum depth has not be reached and current char is still on page
        if depth < self.maxdepth and start_url[-1] == response.xpath('//div[@id="mw-pages"]/div/div/div[1]/h3/text()').get():
            next_page = response.xpath('//div[@id="mw-pages"]/a[2]') # location of next link
            next_page_text = next_page.xpath("text()").get()
            next_page_link = next_page.xpath("@href").get()
            print("### DEBUG DUMP next_page:", next_page, "END ###")

            if next_page_link is not None:
                request = response.follow(next_page_link, callback=self.parse)
                # Meta information: URL of the current page
                request.meta['from'] = response.url
                # Meta information: text of the link
                request.meta['text'] = next_page_text
                # Meta information: depth of the link
                request.meta['depth'] = depth + 1
                # Meta information: start page for current crawler
                request.meta['start'] = start_url
                yield request

    # get article content - using scrapy itemLoader and Items
    def parse_article(self, response):
        l = ItemLoader(item=articleItem(), response=response) # create itemloader l - following is adding to Fields
        l.add_xpath('title',        '//*[@id="firstHeading"]/text()')
        l.add_xpath('publish_date', '//div[@id="catlinks"]/div[@id="mw-normal-catlinks"]/ul/li/a/text()',re='(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)[\s,]*(?:\d{1,2})[\s,]*(?:\d{4})')
        l.add_xpath('content',      '//div[@id="mw-content-text"]/div[@class="mw-parser-output"]/p/text()|//div[@id="mw-content-text"]/div[@class="mw-parser-output"]/p/child::a/text()|//div[@id="mw-content-text"]/div[@class="mw-parser-output"]/ul/li/text()', Join(' '))
        l.add_xpath('categories',   '//div[@id="catlinks"]/div[@id="mw-normal-catlinks"]/ul/li/a/text()')
        l.add_xpath('sources_url',  '//div[@id="mw-content-text"]/div[@class="mw-parser-output"]/ul/li/span/a/@href')
        l.add_xpath('about_sources_wiki_url', '//div[@id="mw-content-text"]/div[@class="mw-parser-output"]/ul/li/span/i/span/a/@href')
        l.add_value('article_url', response.request.url)
        l.add_value('scraped_at', (datetime.today().strftime('%Y-%m-%d')) )
        yield l.load_item() # could use return/yield - no idea what changesw


The general idea for the final Spider is that we generate based on group_nr our entry-point websites and generate a list of all article urls to follow afterwards we follow all these links we follow a link to the next 200 link-page and start again. By using metadata parameters we can inform the spider about where it has been where it is going and how deep it has gone. The last part about the spider to talk about is the function 'parse_article' where we make use of scrapy's item containers which help us deal with missing data in the case of a broken link or other unforeseen circumstances.

The datafields we ended up collecting were:

'article_url'

'title' = title of the article inside the page

'categories' = categories assigned to the article

'content' = main text of article

'publish_date'

'scraped_at' = date of scraping by our spider

'sources_url' = urls for all individual pages used as a source

'about_sources_wiki_url' = url to wikipage about a given source ie. BBC


We felt that these would be of use for further analyses for another group as well as being general enough that most articles would have an entry for all fields.

